In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.pure_RMT import PureRMT
from lib.models.LeeNet.score_pureRMT_mlp import ScorePureRMTMLP
from lib.models.head.mlp import MLP
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import build_dataloaders, get_optimizer_scheduler
from lib.config.cfg_loader import env_setting
from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch


def build_model(cfg):
    backbone = PureRMT(cfg=cfg)
    head = MLP(input_dim=cfg.model.pureRMT.embed_dim[-1], hidden_dim=cfg.model.pureRMT.embed_dim[-1], output_dim=4, num_layers=2)
    model = ScorePureRMTMLP(backbone, head, cfg)
    return model


In [28]:
cfg = env_setting(cfg_name=None)

loader_train, loader_val = build_dataloaders(cfg)

net = build_model(cfg)

focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)

trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

In [29]:
trainer.train(cfg.train.epoch)

x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:05.576912
Avg Data Time: 5.42236
Avg GPU Trans Time: 0.01268
Avg Forward Time: 0.14188
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:05.722600
Avg Data Time: 2.71507
Avg GPU Trans Time: 0.01260
Avg Forward Time: 0.13363
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:05.871433
Avg Data Time: 1.81260
Avg GPU Trans Time: 0.01302
Avg Forward Time: 0.13153
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:06.023059
Avg Data Time: 1.36177
Avg GPU Trans Time: 0.01324
Avg Forward Time: 0.13075
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_

[ WARN:0@8742.168] global loadsave.cpp:248 findDecoder imread_('/home/lzd/dataset/LasHeR/TrainingSet/trainingset/bluegirlriding/visible/v000045.jpg'): can't open/read file: check file path/integrity


x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:11.314170
Avg Data Time: 0.60613
Avg GPU Trans Time: 0.01319
Avg Forward Time: 0.13496
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:11.482571
Avg Data Time: 0.56873
Avg GPU Trans Time: 0.01326
Avg Forward Time: 0.13568


[ WARN:0@8744.190] global loadsave.cpp:248 findDecoder imread_('/home/lzd/dataset/LasHeR/TrainingSet/trainingset/fogboyscoming1_quezhen_inf_heiying/visible/v000399.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8744.805] global loadsave.cpp:248 findDecoder imread_('/home/lzd/dataset/LasHeR/TrainingSet/trainingset/bowblkboy1-quezhen/infrared/i000257.jpg'): can't open/read file: check file path/integrity


x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:13.928473
Avg Data Time: 0.66968
Avg GPU Trans Time: 0.01322
Avg Forward Time: 0.13642
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:14.092041
Avg Data Time: 0.63291
Avg GPU Trans Time: 0.01317
Avg Forward Time: 0.13681
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
Epoch Time: 0:00:14.258765
Avg Data Time: 0.60000
Avg GPU Trans Time: 0.01322
Avg Forward Time: 0.13724
x shape torch.Size([16, 10, 768])
out shape in backbone  torch.Size([16, 10, 768])
pred_boxes shape torch.Size([16, 10, 4])
[train: 1, 20 / 3750] FPS: 22.2 (95.7)  ,  DataTime: 0.570 (0.013)  ,  ForwardTime: 0.138  ,  TotalTime: 0.721  ,  Loss/total: 0.87314  ,  Loss/giou: 0.00000  ,  Loss/l1: 0.43657  ,  Loss/loca

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/lzd/workspace/LeeNet/lib/data/sampler.py", line 98, in __getitem__
    return self.getitem()
  File "/home/lzd/workspace/LeeNet/lib/data/sampler.py", line 212, in getitem
    seq_id, visible, seq_info_dict = self.sample_seq_from_dataset(dataset, is_video_dataset)
  File "/home/lzd/workspace/LeeNet/lib/data/sampler

In [9]:
# from lib.models.layer.patch_embed import PatchEmbed
# from lib.models.backbone.pure_RMT import PureRMT
# import torch
# 
# rgb = None
# modal = None
# 
# device = torch.device("cuda:1")
# embed_dim = 768
# patch_embed = PatchEmbed(patch_size=16, in_chans=3, embed_dim=embed_dim, flatten=False).to(device)
# # relpos = RelPos2d(embed_dim=embed_dim, num_heads=12, initial_value=1, heads_range=3).to(device)
# # ret_block = RetBlock(retention='whole', embed_dim=embed_dim, num_heads=12, ffn_dim=96).to(device)
# # score = ScoreLayerUseConv(embed_dim=embed_dim).to(device)
# template_score = None
# backbone = PureRMT(cfg=cfg).to(device)
# 
# for i, data in enumerate(loader_train, 1):
#     print(i)
#     print(data.keys())
#     # print(data['test_class'])
#     # print(data['dataset'])
#     print("template_images", data['template_images'].shape)
#     print("search_images", data['search_images'].shape)
#     print("search_anno", data['search_anno'].shape)
# 
#     template = data['template_images'][0].to(device)
#     search = data['search_images'][0].to(device)
#     # template, template_patch_num = patch_embed(template)
#     # print("template", template.shape)
#     # positive, uncertain, negative = score(template)
#     # template = template.permute(0,2,3,1) # B,C,H,W -> B,H,W,C
#     backbone(template, search)
# 
#     break

[ WARN:0@5554.715] global loadsave.cpp:248 findDecoder imread_('/home/lzd/dataset/LasHeR/TrainingSet/trainingset/motowithgood/visible/v000000.jpg'): can't open/read file: check file path/integrity


1
odict_keys(['template_images', 'template_anno', 'search_images', 'search_anno', 'dataset', 'test_class', 'valid'])
template_images torch.Size([1, 16, 6, 128, 128])
search_images torch.Size([1, 16, 6, 320, 320])
search_anno torch.Size([1, 16, 4])
x shape torch.Size([16, 10, 768])


In [10]:
# print(template_score.shape)
# print(template_score)